# Day 3 - line segment intersections

* https://adventofcode.com/2019/day/3

This is another [computational geometry](https://en.wikipedia.org/wiki/Computational_geometry) problem; find all the line segment intersections between two sets of lines!

I could go and break open a full [Bently-Ottman algorithm](https://en.wikipedia.org/wiki/Bentley%E2%80%93Ottmann_algorithm) implementation here, but that'd be overkill. First of all, we only need to find where two *distinct* sets of lines need to intersect, and second, all segments are either strictly horizontal, or vertical. The intersections are only going to be found where a horizontal segment crosses a vertical segment.

## Approach

So my approach is to take all the segments of one of the two lines, and see where they cross the horizontal segments of the other if vertical, and the vertical if they are themselves horizontal. We could use fancy binary trees or bisection to optimize finding the matching segments in the series, but I think grouping the segments by `x` (horizontal) or `y` (vertical) coordinate is plenty.

Then all we have to do is:

* classify segments as horizontal or vertical, and record their fixed value and the extent (range) of their start and stop values in the other direction.
* iterate over the segments of one line, A
  * loop over the `extent` values from start to end, as `pos`
      * find all segments on the other line, of the opposite orientation whose extent matches `pos`, B
           * if `fixed` of A is inside the `extent` of B, record an intersection point.

In [1]:
import aocd
data = aocd.get_data(day=3, year=2019)

In [2]:
from __future__ import annotations
from enum import Enum
from dataclasses import dataclass
from typing import Dict, Iterable, NamedTuple, MutableSet, Tuple

class Pos(NamedTuple):
    """Position on the grid"""
    x: int
    y: int
        
    def for_dir(self, dir: Dir) -> int:
        """Get the fixed value for a given direction"""
        return getattr(self, dir.value)
    
    @property
    def distance(self):
        """Manhattan distance to the origin"""
        return sum(map(abs, self))
    
    @classmethod
    def from_fixed_var(cls, fixed: int, var: int, dir: Direction) -> Pos:
        """Create a new position given a fixed and variable value, and direction"""
        v = cls(var, fixed)
        return cls(v.for_dir(dir), v.for_dir(~dir))


class Dir(Enum):
    """Directions on the grid"""
    # value is the position attribute that is fixed
    v = 'x'
    h = 'y'
    
    @classmethod
    def from_direction(cls, direction: str) -> Dir:
        """Direction for a given string UDLR"""
        return cls.v if direction in 'UD' else cls.h
    
    def __invert__(self) -> Dir:
        return Dir.v if self is Dir.h else Dir.h
    

@dataclass(frozen=True)
class Segment:
    dir: Dir
    fixed: int
    extent: range
        
    @classmethod
    def from_direction(cls, from_: Pos, direction: str, length: int) -> [Segment, Pos]:
        """Given a position, direction and length, returns a segment and new position"""
        dir = Dir.from_direction(direction)
        start, fixed = map(from_.for_dir, (dir, ~dir))
        length += 1  # ranges are exclusive, we want inclusive
        start, end = start, start + length
        newpos = Pos.from_fixed_var(fixed, end - 1, dir)
        if direction in 'DL':  # reverse direction
            start, end = (start - length, end - length)
            newpos = Pos.from_fixed_var(fixed, start + 1, dir)
        return cls(dir, fixed, range(start, end)), newpos

LineMap = Dict[Dir, Dict[int, MutableSet[Segment]]]

def segments_from_directions(directions: Iterable[str]) -> Iterable[Segment]:
    pos = Pos(0, 0)
    for dir_length in directions:
        seg, pos = Segment.from_direction(pos, dir_length[0], int(dir_length[1:]))
        yield seg


def create_line_map(segments: Iterable[Segment]) -> LineMap:
    map = {}
    for seg in segments:
        map.setdefault(seg.dir, {}).setdefault(seg.fixed, set()).add(seg)
    return map


def intersections(line_map: LineMap, segments: Iterable[Segment]) -> Iterable[int]:
    for seg in segments:
        opposite_segments = line_map[~seg.dir]
        for pos in seg.extent:
            if pos not in opposite_segments:
                continue
            for opposite in opposite_segments[pos]:
                if seg.fixed in opposite.extent:
                    pos = Pos.from_fixed_var(seg.fixed, pos, seg.dir)
                    distance = pos.distance
                    if distance:
                        yield distance


def min_distance(lines: str) -> int:
    line_a, line_b = (l.split(',') for l in lines.splitlines())
    line_map = create_line_map(segments_from_directions(line_a))
    distances = intersections(line_map, segments_from_directions(line_b))
    return min(distances)

    
tests = {
    "R8,U5,L5,D3\nU7,R6,D4,L4": 6,
    (
        "R75,D30,R83,U83,L12,D49,R71,U7,L72\n"
        "U62,R66,U55,R34,D71,R55,D58,R83"
    ): 159,
    (
        "R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51\n"
        "U98,R91,D20,R16,D67,R40,U7,R15,U6,R7"
    ): 135,
}
for lines, expected in tests.items():
    result = min_distance(lines)
    assert result == expected

In [3]:
print("Part 1:", min_distance(data))

Part 1: 2180
